In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import csv
import janitor
import numpy as np

In [2]:
# sample list
mapping_16S = pd.read_csv("~/Documents/thesis/16S/qiime2/mappingfiles/16S_mapping.csv")
mapping_16S.head()

,#SampleID,Mother_Child,SubjectID,MaternalID,TimePoint,Mgx_batch Mapping,16S_batch Mapping
0,C0052-5F-1A,C,52,0052_m,5,Mgx_batch001,16S_batch001
1,C0126-8F-1A,C,126,0126_m,8,Mgx_batch001,16S_batch001
2,C0126-8F-1B,C,126,0126_m,8,Mgx_batch001,16S_batch001
3,C0127-6F-1B,C,127,0127_m,6,Mgx_batch001,16S_batch001
4,C0286-5F-1A,C,286,0286_m,5,Mgx_batch001,16S_batch001


In [3]:
samplenames = list(mapping_16S["#SampleID"])

In [4]:
len(samplenames)

345

In [5]:
# read in mgx data
mgx = pd.read_csv("mgx_genera.csv")
mgx.rename(columns={'Taxon':'taxa'}, inplace=True)
mgx.head()


,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
0,Acidaminococcaceae_unclassified,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
1,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.002632,0.0,0.0,0.000000,0.03501,0.0,0.0
2,Acinetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
3,Actinobacillus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000139,0.000000,0.0,0.0,0.000116,0.00000,0.0,0.0
4,Actinobaculum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0


In [6]:
# ratio of unclassified organisms profiled with mgx
sum(mgx.taxa.str.contains("_unclassified| noname"))/mgx.shape[0]

0.041916167664670656

In [7]:
# remove taxa that are unclassified or have no name
# "_unclassified"
# "_noname"
mgx = mgx[~mgx.taxa.str.contains("_unclassified")]
mgx = mgx[~mgx.taxa.str.contains("_noname")]

In [8]:
mgx.sample(15)

,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
150,Sphingobacterium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
32,Butyrivibrio,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
156,Sutterella,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.019132,0.000000,0.000000,0.000360,0.016596,0.000000,0.014343,0.018513
77,Gemella,0.000000,0.000000,0.000000,0.000000,0.000000,0.000290,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
65,Enterococcus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003034,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Adlercreutzia,0.000450,0.001152,0.001519,0.001878,0.000000,0.000000,0.000239,0.000945,0.000000,...,0.000357,0.0,0.000000,0.000984,0.000000,0.000000,0.001534,0.000078,0.001053,0.000703
46,Collinsella,0.021758,0.000000,0.000000,0.000000,0.003368,0.032115,0.014475,0.015372,0.000000,...,0.012545,0.0,0.000000,0.004446,0.000000,0.003000,0.011440,0.004458,0.009387,0.005104
71,Faecalibacterium,0.113878,0.116234,0.117554,0.146072,0.098546,0.037753,0.182509,0.065927,0.000000,...,0.095782,0.0,0.063018,0.073613,0.088445,0.035019,0.064412,0.049987,0.076631,0.052125
56,Dialister,0.045603,0.000000,0.000000,0.000000,0.001748,0.000000,0.000000,0.016098,0.000000,...,0.008597,0.0,0.000000,0.000000,0.000000,0.000000,0.026304,0.002382,0.011484,0.000000
15,Anaeroglobus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
mgx.to_csv('mgx_abundance.csv', index=False)

In [10]:
# read in 16S data
amplicon = pd.read_csv("feature-table.csv")
amplicon.rename(columns={'#OTU ID':'taxa'}, inplace=True)


In [11]:
# clean taxonomic names
# keep only genera
amplicon["taxa"] = amplicon['taxa'].str.split("D_5__", expand=True)[1]
amplicon.dropna(inplace=True)


In [12]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
0,Methanobrevibacter,0,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,19,0,0,0
7,Subgroup 10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Subgroup 23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Ilumatobacter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Actinomyces,0,0,7,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
len(amplicon.columns.values)

264

In [14]:
amplicon.sort_values(by = "taxa", ascending=True, inplace=True)

In [15]:
amplicon["taxa"] = amplicon["taxa"].astype(str)

In [16]:
# ratio of unclassified genera for 16S profiling

sum(amplicon.taxa.str.contains("_unclassified| unidentified| uncultured| group"))/amplicon.shape[0]

0.06435643564356436

In [17]:
# remove any Nones, uncultured organisms
amplicon = amplicon[~amplicon.taxa.str.contains("uncultured")]
amplicon = amplicon[~amplicon.taxa.str.contains("metagenome")]
amplicon = amplicon[~amplicon.taxa.str.contains("unidentified")]
amplicon = amplicon[~amplicon.taxa.str.contains(r'[0-9]')]
amplicon = amplicon[~amplicon.taxa.str.contains("group")]

In [18]:
# calculate relative abundances
bug_sum = amplicon.sum(axis = 0, skipna = True)
amplicon.iloc[:, 1:] = amplicon.iloc[:, 1:]/bug_sum

In [19]:
amplicon.to_csv('16S_abundance.csv')

### looking for the intersection of taxonomy

In [20]:
# calculate number of taxa in both vs. either one

In [21]:
amplicon_taxa_list = set(amplicon["taxa"])
mgx_taxa_list = set(mgx["taxa"])

In [22]:
intersection = [bug for bug in amplicon_taxa_list if bug in mgx_taxa_list]

def union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

union = union(list(set(amplicon_taxa_list)), list(set(mgx_taxa_list)))


In [23]:
union_df = pd.DataFrame(union)
union_df.to_csv('taxalist.csv', index=False)

In [24]:
len(intersection)
len(amplicon_taxa_list)-len(intersection)
len(mgx_taxa_list)-len(intersection)

84

159

59

In [25]:
amplicon_only = [bug for bug in amplicon_taxa_list if bug not in intersection]
mgx_only = [bug for bug in mgx_taxa_list if bug not in intersection]


In [26]:
len(amplicon_only)
len(mgx_only)

159

59

### creating long-form dataframe with data from both sequencing methods

In [27]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
245,Acetanaerobacterium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
335,Acetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438,Acholeplasma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,Achromobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011094,0.009139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# only keeping samples we have both mgx and 16S profiles for
mgx_unique = [sample for sample in list(mgx.columns.values) if sample not in list(amplicon.columns.values)] 
amplicon_unique = [sample for sample in list(amplicon.columns.values) if sample not in list(mgx.columns.values)]

In [29]:
mgx.drop(mgx_unique, axis=1, inplace=True)
amplicon.drop(amplicon_unique, axis=1, inplace=True)

In [30]:
# confirm that we now have the same number of samples
len(mgx.columns.values)
len(amplicon.columns.values)

262

262

In [31]:
amplicon_melt = pd.melt(amplicon, id_vars=["taxa"], var_name = "sampleid", value_name = "amplicon_abund")

In [32]:
mgx_melt = pd.melt(mgx, id_vars=["taxa"], var_name = "sampleid", value_name = "mgx_abund")

In [33]:
mgx_melt.head()

,taxa,sampleid,mgx_abund
0,Acidaminococcus,M0652-1F-1A,0.0
1,Acinetobacter,M0652-1F-1A,0.0
2,Actinobacillus,M0652-1F-1A,0.0
3,Actinobaculum,M0652-1F-1A,0.0
4,Actinomyces,M0652-1F-1A,0.0


In [34]:
amplicon_melt.head()

,taxa,sampleid,amplicon_abund
0,Acetanaerobacterium,C0047-7E-1A,0.0
1,Acetobacter,C0047-7E-1A,0.0
2,Acholeplasma,C0047-7E-1A,0.0
3,Achromobacter,C0047-7E-1A,0.0
4,Acidaminococcus,C0047-7E-1A,0.0


In [35]:
merged_taxa = pd.merge(amplicon_melt, mgx_melt, on = ["sampleid", "taxa"], how = "outer")

In [36]:
merged_taxa["abs_diff"] = abs(merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])
merged_taxa["tot_diff"] = (merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])

In [37]:
merged_taxa.sample(10)

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
27978,Bacillus,M0676-1E-1A,0.000000,NaN,NaN,NaN
5059,Romboutsia,C0451-2F-1A,0.006578,NaN,NaN,NaN
53476,Allisonella,M1007-1F-1A,0.000567,NaN,NaN,NaN
3584,Peptostreptococcus,C0286-6E-1A,0.000000,0.000000,0.0,0.0
73523,Clostridium,C0752-4F-1A,NaN,0.018308,NaN,NaN
77481,Dolosigranulum,M1064-2F-1A,NaN,0.000000,NaN,NaN
26772,Butyricimonas,M0656-2F-1A,0.000000,0.000000,0.0,0.0
37405,Sulfurospirillum,M0752-3F-1A,0.000000,NaN,NaN,NaN
50659,Hildenbrandia rubra,M0961-3F-1A,0.000000,NaN,NaN,NaN
48675,Draconibacterium,M0947-2F-1A,0.000000,NaN,NaN,NaN


In [38]:
merged_taxa.to_csv('taxa_abundance_comparison.csv')

In [45]:
amplicon_avg_abund = merged_taxa.groupby("taxa")["amplicon_abund"].mean()
mgx_avg_abund = merged_taxa.groupby("taxa")["mgx_abund"].mean()
taxa_list = sorted(set(merged_taxa["taxa"]))

In [46]:
mean_taxa_abund = pd.DataFrame(
    (zip(taxa_list, amplicon_avg_abund, mgx_avg_abund)),  
    columns = ['taxa','amp_avg_abund', 'mgx_avg_abund'])

In [47]:
mean_taxa_abund["abs_diff"] = abs(mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"])
mean_taxa_abund["total_diff"] = mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"]

In [48]:
mean_taxa_abund.sort_values("abs_diff", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [49]:
mean_taxa_abund.sample(10)

,taxa,amp_avg_abund,mgx_avg_abund,abs_diff,total_diff
0,Acetanaerobacterium,0.000007,NaN,NaN,NaN
20,Alloscardovia,NaN,0.000016,NaN,NaN
7,Actinobacillus,0.000005,0.000006,0.000001,-0.000001
93,Dubosiella,0.000044,NaN,NaN,NaN
251,Rubripirellula,0.000070,NaN,NaN,NaN
301,endosymbionts,0.000005,NaN,NaN,NaN
41,Bilophila,0.002007,0.003580,0.001573,-0.001573
226,Planktomarina,0.000036,NaN,NaN,NaN
188,Methanobrevibacter,0.000209,0.002146,0.001937,-0.001937
160,Lactobacillus,0.002113,0.001776,0.000337,0.000337


In [50]:
mean_taxa_abund.to_csv('taxa_difference.csv')

## making giant dataframe of abundances

In [52]:
amp_trans = amplicon.set_index("taxa").transpose()

In [53]:
amp_trans.reset_index(level=0, inplace=True)

In [54]:
amp_trans.rename(columns = {'index':'sampleid'}, inplace = True) 

In [55]:
amp_trans["uid"] = amp_trans["sampleid"].astype(str)+'-amp'

In [56]:
amp_trans["method"] = "amp"

In [57]:
amp_trans.head()

taxa,sampleid,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,Actibacter,Actinobacillus,Actinomyces,...,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts,uid,method
0,C0047-7E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0047-7E-1A-amp,amp
1,C0052-7E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0052-7E-1A-amp,amp
2,C0053-8E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000888,...,0.000634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0053-8E-1A-amp,amp
3,C0058-4E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.003929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0058-4E-1A-amp,amp
4,C0059-4E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.001789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0059-4E-1A-amp,amp


In [59]:
mgx_trans = mgx.set_index("taxa").transpose()

In [60]:
mgx_trans.reset_index(level=0, inplace=True)

In [61]:
mgx_trans.rename(columns = {'index':'sampleid'}, inplace = True)

In [62]:
mgx_trans["uid"] = mgx_trans["sampleid"].astype(str)+'-mgx'

In [63]:
mgx_trans["method"] = "mgx"

In [64]:
mgx_trans.head()

taxa,sampleid,Acidaminococcus,Acinetobacter,Actinobacillus,Actinobaculum,Actinomyces,Adlercreutzia,Aeromonas,Aggregatibacter,Akkermansia,...,Tannerella,Treponema,Tunalikevirus,Turicibacter,Ureaplasma,Varibaculum,Veillonella,Weissella,uid,method
0,M0652-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000131,0.0,0.0,0.001095,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0652-1F-1A-mgx,mgx
1,M0647-1F-1A,0.0,0.0,0.0,0.0,0.0,0.001504,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.001681,0.0,0.0,0.001841,0.0,M0647-1F-1A-mgx,mgx
2,M0656-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000907,0.0,0.0,0.019614,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0656-1F-1A-mgx,mgx
3,C0660-1E-1A,0.0,0.0,0.0,0.0,0.0,0.000575,0.0,0.0,0.005417,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000314,0.0,C0660-1E-1A-mgx,mgx
4,M0649-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0649-1F-1A-mgx,mgx


In [65]:
concat_df = pd.concat([mgx_trans,amp_trans], sort=True).reset_index(drop = True)

In [82]:
len(concat_df)

198

In [67]:
age = pd.read_csv("~/Documents/thesis/analysis/metadatawide.csv")

/Users/danielle/miniconda2/envs/thesis_analysis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (22,23,99,110,152,212,252,291,294,321,326,415,440,540,547,574,577) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
age = age[['sample','childAgeMonths']]

In [69]:
age["sample"] = age["sample"].str.replace("_",'-')
age.rename(columns = {'sample':'sampleid'}, inplace = True) 

In [70]:
# make age dictionary
agedict = {str(s): {} for s in age["sampleid"]}
for index, row in age.iterrows():
    age_months = row["childAgeMonths"]
    agedict[row["sampleid"]]= age_months

In [71]:
concat_df["AgeMonths"]= concat_df["sampleid"].map(agedict)

In [73]:
# remove mothers
concat_df = concat_df[~concat_df.sampleid.str.contains("M")]

In [74]:
len(concat_df)

198

In [77]:
cols_to_order = ['uid', 'sampleid',"method", "AgeMonths"]
new_columns = cols_to_order + (concat_df.columns.drop(cols_to_order).tolist())
concat_df = concat_df[new_columns]

In [78]:
concat_df.sample(15)

,uid,sampleid,method,AgeMonths,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,...,Ureaplasma,Varibaculum,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts
344,C0851-4F-1A-amp,C0851-4F-1A,amp,12.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.021706,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
230,C0936-1F-1A-mgx,C0936-1F-1A,mgx,28.0,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.0,0.0,0.000221,NaN,NaN,NaN,0.0,NaN,NaN,NaN
108,C0494-2E-1A-mgx,C0494-2E-1A,mgx,44.0,NaN,NaN,NaN,NaN,0.000000,0.000000,...,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN
271,C0192-4F-1A-amp,C0192-4F-1A,amp,100.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.002257,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
265,C0059-4E-1A-amp,C0059-4E-1A,amp,157.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.001789,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
322,C0739-2F-1A-amp,C0739-2F-1A,amp,17.0,0.000000,0.0,0.0,0.0,0.000126,0.006064,...,NaN,NaN,0.151071,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
274,C0237-4E-1A-amp,C0237-4E-1A,amp,106.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.001949,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
262,C0052-7E-1A-amp,C0052-7E-1A,amp,143.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.000988,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
284,C0494-2E-1A-amp,C0494-2E-1A,amp,44.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
345,C0925-1F-1A-amp,C0925-1F-1A,amp,43.0,0.000127,0.0,0.0,0.0,0.000000,0.000000,...,NaN,NaN,0.000509,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [79]:
concat_df.to_csv('transposed_mgxamp_df.csv')

In [80]:
pwd()

'/Users/danielle/Documents/thesis/analysis'